In [1]:
import pandas as pd
import requests
import os
from datetime import datetime

FRED_API_KEY = os.getenv('FRED_API_KEY')
BASE_URL = 'https://api.stlouisfed.org/fred'

series_list = ['CPIAUCSL', 'CPILFESL', 'CPIUFDSL', 'CPIHOSSL', 
                'CUSR0000SETB01', 'PCEPI', 'PCEPILFE', 
                'DPCERD3A086NBEA', 'GASREGW', 'DCOILWTICO']

series_id = 'CPIAUCSL'
start_date = '2019-01-01'

In [2]:
# 1. 시계열 데이터 다운로드
obs_url = f'{BASE_URL}/series/observations'
obs_params = {
    'api_key': FRED_API_KEY,
    'file_type': 'json',
    'series_id': series_id,
    'observation_start': start_date,
    'observation_end': datetime.now().strftime('%Y-%m-%d')
}
obs_data = requests.get(obs_url, params=obs_params).json()
obs_df = pd.DataFrame(obs_data['observations'])
obs_df = obs_df[obs_df['value'] != '.']
obs_df['date'] = pd.to_datetime(obs_df['date'])
obs_df['value'] = pd.to_numeric(obs_df['value'])

# realtime_start, realtime_end drop
obs_df = obs_df.drop(columns=['realtime_start', 'realtime_end'])
obs_df['Date'] = obs_df['date']
obs_df = obs_df.drop(columns=['date'])

# Relase_date 는 YYYY-MM 으로 변환
obs_df['Date'] = obs_df['Date'].dt.strftime('%Y-%m')
obs_df

# 한 칸씩 MM을 더하기
obs_df['Date'] = obs_df['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m') + pd.DateOffset(months=1))
obs_df

,value,Date
0,252.561,2019-02-01
1,253.319,2019-03-01
2,254.277,2019-04-01
3,255.233,2019-05-01
4,255.296,2019-06-01
...,...,...
69,315.564,2024-11-01
70,316.449,2024-12-01
71,317.603,2025-01-01
72,319.086,2025-02-01


In [ ]:
# 2. release_id 추출 및 release_dates 수집
release_url = f'{BASE_URL}/series/release'
release_params = {'api_key': FRED_API_KEY, 'file_type': 'json', 'series_id': series_id}
release_id = requests.get(release_url, params=release_params).json()['releases'][0]['id']
rel_url = f'{BASE_URL}/release/dates'

rel_params = {'api_key': FRED_API_KEY, 'file_type': 'json', 'release_id': release_id}
rel_data = requests.get(rel_url, params=rel_params).json()['release_dates']
rel_df = pd.DataFrame(rel_data)
rel_df['release_date'] = pd.to_datetime(rel_df['date'])
rel_df = rel_df.sort_values('release_date')
rel_df['Date'] = rel_df['release_date'].dt.strftime('%Y-%m')
rel_df = rel_df.drop(columns=['date'])
rel_df

,release_id,release_date,Date
0,10,1949-03-24,1949-03
1,10,1949-04-22,1949-04
2,10,1949-05-23,1949-05
3,10,1949-06-24,1949-06
4,10,1949-07-22,1949-07
...,...,...,...
927,10,2024-11-13,2024-11
928,10,2024-12-11,2024-12
929,10,2025-01-15,2025-01
930,10,2025-02-12,2025-02


In [4]:
# Date 와 Release_Date 를 합치기
obs_df = pd.merge(obs_df, rel_df, on='Date', how='left')
obs_df

ValueError: You are trying to merge on datetime64[ns] and object columns for key 'Date'. If you wish to proceed you should use pd.concat

In [18]:
import pandas as pd
import requests
import os
from datetime import datetime

FRED_API_KEY = os.getenv('FRED_API_KEY')
BASE_URL = 'https://api.stlouisfed.org/fred'
series_id = 'CPIAUCSL'
start_date = '2019-01-01'

# 1. 관측값 다운로드
obs_url = f'{BASE_URL}/series/observations'
obs_params = {
    'api_key': FRED_API_KEY,
    'file_type': 'json',
    'series_id': series_id,
    'observation_start': start_date,
    'observation_end': datetime.now().strftime('%Y-%m-%d')
}
obs_data = requests.get(obs_url, params=obs_params).json()
obs_df = pd.DataFrame(obs_data['observations'])
obs_df = obs_df[obs_df['value'] != '.']
obs_df['date'] = pd.to_datetime(obs_df['date'])
obs_df['value'] = pd.to_numeric(obs_df['value'])
# 나머지 drop
obs_df = obs_df.drop(columns=['realtime_start', 'realtime_end'])
obs_df['Date'] = obs_df['date']
obs_df = obs_df.drop(columns=['date'])

# 2. release_id → release_dates 가져오기
meta_url = f'{BASE_URL}/series/release'
meta_params = {
    'api_key': FRED_API_KEY,
    'file_type': 'json',
    'series_id': series_id
}
release_id = requests.get(meta_url, params=meta_params).json()['releases'][0]['id']

release_url = f'{BASE_URL}/release/dates'
release_params = {
    'api_key': FRED_API_KEY,
    'file_type': 'json',
    'realtime_start' : start_date,
    'release_id': release_id
}
release_data = requests.get(release_url, params=release_params).json()['release_dates']
rel_df = pd.DataFrame(release_data)
rel_df['release_date'] = pd.to_datetime(rel_df['date'])
rel_df = rel_df.drop(columns=['date'])
rel_df = rel_df.sort_values('release_date').reset_index(drop=True)

In [19]:
obs_df

,value,Date
0,252.561,2019-01-01
1,253.319,2019-02-01
2,254.277,2019-03-01
3,255.233,2019-04-01
4,255.296,2019-05-01
...,...,...
69,315.564,2024-10-01
70,316.449,2024-11-01
71,317.603,2024-12-01
72,319.086,2025-01-01


In [20]:
rel_df

,release_id,release_date
0,10,2019-01-11
1,10,2019-02-11
2,10,2019-02-13
3,10,2019-03-12
4,10,2019-04-10
...,...,...
76,10,2024-11-13
77,10,2024-12-11
78,10,2025-01-15
79,10,2025-02-12
